In [38]:
import numpy as np
import pandas as pd
import os
import sys
import datetime

## 1 读取预报文件

In [39]:
targetpath=r"D:\git仓库\GridForecastSys\extra_data\grid_data"
# filename="Sample_new.txt"
filename="BasicUnit_20180709.txt"
fullname=os.path.join(targetpath,filename)
print(fullname)
#data=np.loadtxt(fullname)
# data=pd.read_table(targetpath,encoding = 'utf-8',header=None)
with open(fullname,"rb") as f:
#     columns_date=pd.read_table(f,sep='\t',encoding='utf-8',nrows=1,header=None)
    data=pd.read_table(f,sep='\t',encoding='utf-8',header=None,infer_datetime_format=True)
    print('读取成功')

D:\git仓库\GridForecastSys\extra_data\grid_data\BasicUnit_20180709.txt
读取成功


In [40]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,NaN,NaN,2018-07-09 08:00,2018-07-09 09:00,2018-07-09 10:00,2018-07-09 11:00,2018-07-09 12:00,2018-07-09 13:00,2018-07-09 14:00,2018-07-09 15:00,...,2018-07-11 23:00,2018-07-12 00:00,2018-07-12 01:00,2018-07-12 02:00,2018-07-12 03:00,2018-07-12 04:00,2018-07-12 05:00,2018-07-12 06:00,2018-07-12 07:00,2018-07-12 08:00
1,N01,HS,0.36,0.35,0.34,0.33,0.33,0.32,0.31,0.31,...,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.57,0.57
2,N01,PER,3.43,3.58,3.73,3.86,3.97,4.07,4.16,4.25,...,3.53,3.51,3.49,3.48,3.49,3.51,3.54,3.55,3.57,3.59
3,N01,DIR,0.07,1.39,3.01,4.49,5.76,7.04,8.33,9.58,...,16.77,16.56,16.55,16.68,17.20,17.93,18.66,19.26,19.82,20.37
4,N02,HS,0.40,0.40,0.40,0.39,0.38,0.38,0.38,0.39,...,0.65,0.65,0.66,0.68,0.68,0.68,0.68,0.69,0.69,0.70


#### 读取第一行
 注意此时的第一行数据为一个series

In [41]:
columns_date=data.iloc[0]
columns_date.head()

0                 NaN
1                 NaN
2    2018-07-09 08:00
3    2018-07-09 09:00
4    2018-07-09 10:00
Name: 0, dtype: object

### 数据清理
#### 处理nan
 对于series仍可以使用.notnull()与.isnull()过滤nan  
 返回的类型均为series

* 此处可以使用

#### 1-2 df.dropna(axis=1)：删除所有包含空值的列  
 注意对于Series类型的数据不能使用.dropna（）方法

In [42]:
# 转成list
columns_date_notnan=columns_date[columns_date.notnull()]
columns_date_notnan.head()
# columns_date_convert

2    2018-07-09 08:00
3    2018-07-09 09:00
4    2018-07-09 10:00
5    2018-07-09 11:00
6    2018-07-09 12:00
Name: 0, dtype: object

#### series的values是一个numpy.ndarray

In [23]:
print(columns_date_notnan.values[0])
print(type(columns_date_notnan.values[0]))

2018-07-09 08:00
<class 'str'>


调用map方法  
 还需去掉.0
 #### 最终采取此种方式：
 * 将第一行使用map转成int类型（去掉了.0）  
 * 再转换为str类型

In [24]:
columns_date_notnan_convert=columns_date_notnan
# 3月7 日修改——时间变量由yyyymmddhh改为yyyy-mm-dd hh:00的这种方式
# columns_date_notnan_convert=columns_date_notnan.map(int).map(str)
print(columns_date_notnan_convert.values[0])
print(type(columns_date_notnan_convert.values[0]))

2018-07-09 08:00
<class 'str'>


## 3 读取除去第一行的其余数据

In [43]:
data=data[1:]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
1,N01,HS,0.36,0.35,0.34,0.33,0.33,0.32,0.31,0.31,...,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.57,0.57
2,N01,PER,3.43,3.58,3.73,3.86,3.97,4.07,4.16,4.25,...,3.53,3.51,3.49,3.48,3.49,3.51,3.54,3.55,3.57,3.59
3,N01,DIR,0.07,1.39,3.01,4.49,5.76,7.04,8.33,9.58,...,16.77,16.56,16.55,16.68,17.20,17.93,18.66,19.26,19.82,20.37
4,N02,HS,0.40,0.40,0.40,0.39,0.38,0.38,0.38,0.39,...,0.65,0.65,0.66,0.68,0.68,0.68,0.68,0.69,0.69,0.70
5,N02,PER,3.65,3.73,3.84,3.93,3.98,4.00,4.00,4.01,...,3.98,3.93,3.90,3.86,3.87,3.90,3.94,3.97,3.97,3.98


## 4 更改索引

#### 此种方法才是更改索引

In [44]:
data=data.set_index([0,1])

In [45]:
data.head()

2     3     4     5     6     7     8     9      10     11  ...    \
0   1                                                                  ...     
N01 HS   0.36  0.35  0.34  0.33  0.33  0.32  0.31  0.31   0.31   0.32  ...     
    PER  3.43  3.58  3.73  3.86  3.97  4.07  4.16  4.25   4.31   4.32  ...     
    DIR  0.07  1.39  3.01  4.49  5.76  7.04  8.33  9.58  11.00  12.81  ...     
N02 HS   0.40  0.40  0.40  0.39  0.38  0.38  0.38  0.39   0.39   0.39  ...     
    PER  3.65  3.73  3.84  3.93  3.98  4.00  4.00  4.01   4.05   4.10  ...     

            65     66     67     68     69     70     71     72     73     74  
0   1                                                                          
N01 HS    0.56   0.56   0.56   0.56   0.57   0.57   0.57   0.57   0.57   0.57  
    PER   3.53   3.51   3.49   3.48   3.49   3.51   3.54   3.55   3.57   3.59  
    DIR  16.77  16.56  16.55  16.68  17.20  17.93  18.66  19.26  19.82  20.37  
N02 HS    0.65   0.65   0.66   0.68   0.68   0.68   0.68   0.69   0.69   0.70  
    PER   3.98   3.93   3.90   3.86   3.87   3.90   3.94   3.97   3.97   3.98  

[5 rows x 73 columns]

## 5 修改head

In [46]:
data.columns=columns_date_notnan_convert

In [47]:
data.head()

0       2018-07-09 08:00 2018-07-09 09:00 2018-07-09 10:00 2018-07-09 11:00  \
0   1                                                                         
N01 HS              0.36             0.35             0.34             0.33   
    PER             3.43             3.58             3.73             3.86   
    DIR             0.07             1.39             3.01             4.49   
N02 HS              0.40             0.40             0.40             0.39   
    PER             3.65             3.73             3.84             3.93   

0       2018-07-09 12:00 2018-07-09 13:00 2018-07-09 14:00 2018-07-09 15:00  \
0   1                                                                         
N01 HS              0.33             0.32             0.31             0.31   
    PER             3.97             4.07             4.16             4.25   
    DIR             5.76             7.04             8.33             9.58   
N02 HS              0.38             0.38             0.38             0.39   
    PER             3.98             4.00             4.00             4.01   

0       2018-07-09 16:00 2018-07-09 17:00       ...        2018-07-11 23:00  \
0   1                                           ...                           
N01 HS              0.31             0.32       ...                    0.56   
    PER             4.31             4.32       ...                    3.53   
    DIR            11.00            12.81       ...                   16.77   
N02 HS              0.39             0.39       ...                    0.65   
    PER             4.05             4.10       ...                    3.98   

0       2018-07-12 00:00 2018-07-12 01:00 2018-07-12 02:00 2018-07-12 03:00  \
0   1                                                                         
N01 HS              0.56             0.56             0.56             0.57   
    PER             3.51             3.49             3.48             3.49   
    DIR            16.56            16.55            16.68            17.20   
N02 HS              0.65             0.66             0.68             0.68   
    PER             3.93             3.90             3.86             3.87   

0       2018-07-12 04:00 2018-07-12 05:00 2018-07-12 06:00 2018-07-12 07:00  \
0   1                                                                         
N01 HS              0.57             0.57             0.57             0.57   
    PER             3.51             3.54             3.55             3.57   
    DIR            17.93            18.66            19.26            19.82   
N02 HS              0.68             0.68             0.69             0.69   
    PER             3.90             3.94             3.97             3.97   

0       2018-07-12 08:00  
0   1                     
N01 HS              0.57  
    PER             3.59  
    DIR            20.37  
N02 HS              0.70  
    PER             3.98  

[5 rows x 73 columns]

In [30]:
data.index.names=['','']
data.index.names=['code','factor']

In [31]:
data.head()

0           2018-07-09 08:00 2018-07-09 09:00 2018-07-09 10:00  \
code factor                                                      
N01  HS                 0.36             0.35             0.34   
     PER                3.43             3.58             3.73   
     DIR                0.07             1.39             3.01   
N02  HS                 0.40             0.40             0.40   
     PER                3.65             3.73             3.84   

0           2018-07-09 11:00 2018-07-09 12:00 2018-07-09 13:00  \
code factor                                                      
N01  HS                 0.33             0.33             0.32   
     PER                3.86             3.97             4.07   
     DIR                4.49             5.76             7.04   
N02  HS                 0.39             0.38             0.38   
     PER                3.93             3.98             4.00   

0           2018-07-09 14:00 2018-07-09 15:00 2018-07-09 16:00  \
code factor                                                      
N01  HS                 0.31             0.31             0.31   
     PER                4.16             4.25             4.31   
     DIR                8.33             9.58            11.00   
N02  HS                 0.38             0.39             0.39   
     PER                4.00             4.01             4.05   

0           2018-07-09 17:00       ...        2018-07-11 23:00  \
code factor                        ...                           
N01  HS                 0.32       ...                    0.56   
     PER                4.32       ...                    3.53   
     DIR               12.81       ...                   16.77   
N02  HS                 0.39       ...                    0.65   
     PER                4.10       ...                    3.98   

0           2018-07-12 00:00 2018-07-12 01:00 2018-07-12 02:00  \
code factor                                                      
N01  HS                 0.56             0.56             0.56   
     PER                3.51             3.49             3.48   
     DIR               16.56            16.55            16.68   
N02  HS                 0.65             0.66             0.68   
     PER                3.93             3.90             3.86   

0           2018-07-12 03:00 2018-07-12 04:00 2018-07-12 05:00  \
code factor                                                      
N01  HS                 0.57             0.57             0.57   
     PER                3.49             3.51             3.54   
     DIR               17.20            17.93            18.66   
N02  HS                 0.68             0.68             0.68   
     PER                3.87             3.90             3.94   

0           2018-07-12 06:00 2018-07-12 07:00 2018-07-12 08:00  
code factor                                                     
N01  HS                 0.57             0.57             0.57  
     PER                3.55             3.57             3.59  
     DIR               19.26            19.82            20.37  
N02  HS                 0.69             0.69             0.70  
     PER                3.97             3.97             3.98  

[5 rows x 73 columns]

* 尝试获取最大值

In [15]:
data.T.max().unstack().loc['E63']

factor
DIR    91.65
HS      9.83
PER     9.96
Name: E63, dtype: float64

## 6 存储为csv文件

In [48]:
targetfilename="convert_date.csv"
final_path=os.path.join(targetfilename)
data.to_csv(final_path)
print(final_path)

convert_date.csv


## 读取该文件

** 读取该生成后的文件，并设置联合索引 **

In [49]:
read_data=pd.read_csv('convert_date.csv')

In [34]:
read_data=read_data.set_index(['code','factor'])

In [35]:
read_data.head()

2018-07-09 08:00  2018-07-09 09:00  2018-07-09 10:00  \
code factor                                                         
N01  HS                  0.36              0.35              0.34   
     PER                 3.43              3.58              3.73   
     DIR                 0.07              1.39              3.01   
N02  HS                  0.40              0.40              0.40   
     PER                 3.65              3.73              3.84   

             2018-07-09 11:00  2018-07-09 12:00  2018-07-09 13:00  \
code factor                                                         
N01  HS                  0.33              0.33              0.32   
     PER                 3.86              3.97              4.07   
     DIR                 4.49              5.76              7.04   
N02  HS                  0.39              0.38              0.38   
     PER                 3.93              3.98              4.00   

             2018-07-09 14:00  2018-07-09 15:00  2018-07-09 16:00  \
code factor                                                         
N01  HS                  0.31              0.31              0.31   
     PER                 4.16              4.25              4.31   
     DIR                 8.33              9.58             11.00   
N02  HS                  0.38              0.39              0.39   
     PER                 4.00              4.01              4.05   

             2018-07-09 17:00        ...         2018-07-11 23:00  \
code factor                          ...                            
N01  HS                  0.32        ...                     0.56   
     PER                 4.32        ...                     3.53   
     DIR                12.81        ...                    16.77   
N02  HS                  0.39        ...                     0.65   
     PER                 4.10        ...                     3.98   

             2018-07-12 00:00  2018-07-12 01:00  2018-07-12 02:00  \
code factor                                                         
N01  HS                  0.56              0.56              0.56   
     PER                 3.51              3.49              3.48   
     DIR                16.56             16.55             16.68   
N02  HS                  0.65              0.66              0.68   
     PER                 3.93              3.90              3.86   

             2018-07-12 03:00  2018-07-12 04:00  2018-07-12 05:00  \
code factor                                                         
N01  HS                  0.57              0.57              0.57   
     PER                 3.49              3.51              3.54   
     DIR                17.20             17.93             18.66   
N02  HS                  0.68              0.68              0.68   
     PER                 3.87              3.90              3.94   

             2018-07-12 06:00  2018-07-12 07:00  2018-07-12 08:00  
code factor                                                        
N01  HS                  0.57              0.57              0.57  
     PER                 3.55              3.57              3.59  
     DIR                19.26             19.82             20.37  
N02  HS                  0.69              0.69              0.70  
     PER                 3.97              3.97              3.98  

[5 rows x 73 columns]

** 获取最大值**

In [36]:
read_data.T.max().unstack().loc['E63']

factor
DIR    345.65
HS      10.33
PER     11.07
Name: E63, dtype: float64

### 7、获取最大值

In [105]:
data_date_max=data.T.idxmax(axis=1)

In [106]:
data_date_max.head()

0
2018-07-09 08:00    (E62, PER)
2018-07-09 09:00    (E62, PER)
2018-07-09 10:00    (E77, DIR)
2018-07-09 11:00    (E77, DIR)
2018-07-09 12:00    (E77, DIR)
dtype: object

In [107]:
data_date_max_2=data.T.idxmax(axis=0)

In [108]:
data_date_max_2.ix['E63']

D:\MyPogram\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


factor
HS     2018-07-11 05:00
PER    2018-07-10 09:00
DIR    2018-07-11 03:00
dtype: object

In [144]:
read_date.T.max().loc['E63']

KeyError: 'the label [E63] is not in the [index]'

In [129]:
data.T.max().to_csv('max_convert.csv')

In [49]:
data.T.idxmax(axis=1).head()

0
2018-07-09 08:00    (E62, PER)
2018-07-09 09:00    (E62, PER)
2018-07-09 10:00    (E77, DIR)
2018-07-09 11:00    (E77, DIR)
2018-07-09 12:00    (E77, DIR)
dtype: object

In [48]:
data_max=data.T.idxmax(axis=0)
data_max.head()
data_max.ix['E63']

D:\MyPogram\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


1
HS     2018-07-11 05:00
PER    2018-07-10 09:00
DIR    2018-07-11 03:00
dtype: object

In [34]:
my_date.T.idxmax(axis=0).head()

0    1  
N01  HS     2018-03-08 16:00
     PER    2018-03-10 03:00
     DIR    2018-03-09 06:00
N02  HS     2018-03-10 08:00
     PER    2018-03-10 02:00
dtype: object

** 获取最大值的方式2 **

In [33]:
data.head()

0       2018-07-09 08:00 2018-07-09 09:00 2018-07-09 10:00 2018-07-09 11:00  \
0   1                                                                         
N01 PER             3.43             3.58             3.73             3.86   
    DIR             0.07             1.39             3.01             4.49   
N02 HS              0.40             0.40             0.40             0.39   
    PER             3.65             3.73             3.84             3.93   
    DIR           321.64           320.81           320.54           320.61   

0       2018-07-09 12:00 2018-07-09 13:00 2018-07-09 14:00 2018-07-09 15:00  \
0   1                                                                         
N01 PER             3.97             4.07             4.16             4.25   
    DIR             5.76             7.04             8.33             9.58   
N02 HS              0.38             0.38             0.38             0.39   
    PER             3.98             4.00             4.00             4.01   
    DIR           320.44           320.60           321.52           323.44   

0       2018-07-09 16:00 2018-07-09 17:00       ...        2018-07-11 23:00  \
0   1                                           ...                           
N01 PER             4.31             4.32       ...                    3.53   
    DIR            11.00            12.81       ...                   16.77   
N02 HS              0.39             0.39       ...                    0.65   
    PER             4.05             4.10       ...                    3.98   
    DIR           325.64           327.43       ...                  339.98   

0       2018-07-12 00:00 2018-07-12 01:00 2018-07-12 02:00 2018-07-12 03:00  \
0   1                                                                         
N01 PER             3.51             3.49             3.48             3.49   
    DIR            16.56            16.55            16.68            17.20   
N02 HS              0.65             0.66             0.68             0.68   
    PER             3.93             3.90             3.86             3.87   
    DIR           341.02           342.49           344.28           345.67   

0       2018-07-12 04:00 2018-07-12 05:00 2018-07-12 06:00 2018-07-12 07:00  \
0   1                                                                         
N01 PER             3.51             3.54             3.55             3.57   
    DIR            17.93            18.66            19.26            19.82   
N02 HS              0.68             0.68             0.69             0.69   
    PER             3.90             3.94             3.97             3.97   
    DIR           346.49           346.94           347.33           347.72   

0       2018-07-12 08:00  
0   1                     
N01 PER             3.59  
    DIR            20.37  
N02 HS              0.70  
    PER             3.98  
    DIR           348.07  

[5 rows x 73 columns]

In [36]:
((data.T).max()).ix['E63']

D:\MyPogram\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


1
HS      9.83
PER     9.96
DIR    91.65
dtype: float64

In [43]:
data.T.max().unstack().ix['E63']

D:\MyPogram\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


1
DIR    91.65
HS      9.83
PER     9.96
Name: E63, dtype: float64